In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset directory
dataset_path = "plantvillage_dataset"

# Define constants
IMG_SIZE = 224  # Input image dimensions
CHANNELS = 3  # RGB images
BATCH_SIZE = 32  # Batch size for training

# Initialize data generator with augmentation
data_generator = ImageDataGenerator(
    rescale=1. / 255,  # Normalize pixel values
    validation_split=0.1,  # Use 10% of the data for validation
    rotation_range=30,  # Randomly rotate images by up to 30 degrees
    width_shift_range=0.2,  # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,  # Apply shear transformations
    zoom_range=0.2,  # Randomly zoom in/out
    horizontal_flip=True  # Flip images horizontally
)

# Create training data generator
train_generator = data_generator.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),  # Resize images
    batch_size=BATCH_SIZE,
    subset='training',  # Use for training
    class_mode='categorical'  # Multi-class classification
)

# Create validation data generator
validation_generator = data_generator.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),  # Resize images
    batch_size=BATCH_SIZE,
    subset='validation',  # Use for validation
    class_mode='categorical'  # Multi-class classification
)

print("Data generators are ready!")

Found 27802 images belonging to 23 classes.
Found 3081 images belonging to 23 classes.
Data generators are ready!


In [8]:
# Get class indices
class_indices = train_generator.class_indices

# Print each class and its index
for class_name, class_index in class_indices.items():
    print(f"Class {class_index}: {class_name}")

Class 0: Corn_(maize)___Cercospora_leaf_spot_Gray_leaf_spot
Class 1: Corn_(maize)___Common_rust_
Class 2: Corn_(maize)___Northern_Leaf_Blight
Class 3: Corn_(maize)___healthy
Class 4: Grape___Black_rot
Class 5: Grape___Esca_(Black_Measles)
Class 6: Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Class 7: Grape___healthy
Class 8: Peach___Bacterial_spot
Class 9: Peach___healthy
Class 10: Potato___Early_blight
Class 11: Potato___Late_blight
Class 12: Potato___healthy
Class 13: Tomato___Bacterial_spot
Class 14: Tomato___Early_blight
Class 15: Tomato___Late_blight
Class 16: Tomato___Leaf_Mold
Class 17: Tomato___Septoria_leaf_spot
Class 18: Tomato___Spider_mites Two-spotted_spider_mite
Class 19: Tomato___Target_Spot
Class 20: Tomato___Tomato_Yellow_Leaf_Curl_Virus
Class 21: Tomato___Tomato_mosaic_virus
Class 22: Tomato___healthy


In [9]:
from tensorflow.keras import layers, models

# Initialize the model
model = models.Sequential()

# Add convolutional and pooling layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS)))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

# Additional convolutional block
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

# Flatten layer
model.add(layers.Flatten())

# Dense layer
model.add(layers.Dense(512, activation='relu'))

# Dropout to prevent overfitting
model.add(layers.Dropout(0.5))

# Output layer for multi-class classification
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

# Print model summary
model.summary()


d:\Graduation Project\Model\v2\plantvillage\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    18,874,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 23)             │        11,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,275,095 (73.53 MB)

 Trainable params: 19,275,095 (73.53 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Compile the model
model.compile(
    optimizer='adam',  # Adaptive optimizer
    loss='categorical_crossentropy',  # Loss for multi-class classification
    metrics=['accuracy']  # Track accuracy
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,  # Stop if no improvement after 10 epochs
    restore_best_weights=True  # Restore the best model
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,  # Reduce learning rate by half
    patience=5,  # Trigger if no improvement after 5 epochs
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,  # Number of training steps
    epochs= 100,  # Maximum number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,  # Validation steps
    callbacks=[early_stopping, reduce_lr]  # Use callbacks
)

print("Model training complete!")


Epoch 1/100
868/868 ━━━━━━━━━━━━━━━━━━━━ 1055s 1s/step - accuracy: 0.3175 - loss: 2.2842 - val_accuracy: 0.6377 - val_loss: 1.1603 - learning_rate: 0.0010
Epoch 2/100
868/868 ━━━━━━━━━━━━━━━━━━━━ 44s 49ms/step - accuracy: 0.4375 - loss: 1.7417 - val_accuracy: 0.6299 - val_loss: 1.1981 - learning_rate: 0.0010
Epoch 3/100
868/868 ━━━━━━━━━━━━━━━━━━━━ 982s 1s/step - accuracy: 0.6568 - loss: 1.0634 - val_accuracy: 0.7985 - val_loss: 0.6164 - learning_rate: 0.0010
Epoch 4/100
868/868 ━━━━━━━━━━━━━━━━━━━━ 39s 45ms/step - accuracy: 0.5938 - loss: 1.0615 - val_accuracy: 0.7881 - val_loss: 0.6315 - learning_rate: 0.0010
Epoch 5/100
868/868 ━━━━━━━━━━━━━━━━━━━━ 1110s 1s/step - accuracy: 0.7735 - loss: 0.6943 - val_accuracy: 0.8574 - val_loss: 0.4471 - learning_rate: 0.0010
Epoch 6/100
868/868 ━━━━━━━━━━━━━━━━━━━━ 86s 97ms/step - accuracy: 0.9688 - loss: 0.2241 - val_accuracy: 0.8506 - val_loss: 0.4430 - learning_rate: 0.0010
Epoch 7/100
868/868 ━━━━━━━━━━━━━━━━━━━━ 1064s 1s/step - accuracy: 0.81

In [13]:
# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(
    validation_generator,
    steps=validation_generator.samples // BATCH_SIZE
)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Save the trained model
model.save("plant_village_model.keras")

96/96 ━━━━━━━━━━━━━━━━━━━━ 40s 420ms/step - accuracy: 0.9821 - loss: 0.0400
Validation Accuracy: 98.47%
